# Evaluation of Vabamorf's disambiguation on spellchecker's suggestions

In this small experiment, we use Vabamorf's spellchecker to add _**multiple normalized forms** to the **words layer**_, and examine, how this increased ambiguity affects the quality of Vabamorf's _disambiguation_ in EstNLTK.

We use EstNLTK's version 1.6.4beta (from the commit [52c921eb3d](https://github.com/estnltk/estnltk/tree/52c921eb3d06ebc0976c0dac84bc9b9f72b0491e)), and evaluate tools on the Estonian Web Treebank (EWTB) corpus.

### Load Estonian Web Treebank corpus

You can download the UD format EWTB corpus from here: https://github.com/UniversalDependencies/UD_Estonian-EWT/ (exact commit: [6cd4d14](https://github.com/UniversalDependencies/UD_Estonian-EWT/tree/6cd4d1480c1f3dc89bcdddab56f04dc51bfa8b48)).

In [1]:
eval_data_dir = 'UD_Estonian-EWT-master'

import os, os.path
from ewtb_ud_utils import load_EWTB_ud_file_with_corrections

# Load corpus files with corrections
ud_layer_name = 'ud_syntax'
loaded_texts  = []
for fname in os.listdir( eval_data_dir ):
    if fname.endswith('.conllu'):
        fpath = os.path.join( eval_data_dir, fname )
        text = load_EWTB_ud_file_with_corrections( fpath, ud_layer_name )
        text.meta['file'] = fname
        loaded_texts.append( text )

---

## Vabamorf's analysis and disambiguation (baseline: no spelling suggestions)

### 0. No spelling suggestions + Vabamorf's analysis

In [2]:
# 'morph_0' == VabamorfAnalyzer + PostMorphAnalysisTagger
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_0')
post_corrector = PostMorphAnalysisTagger(output_layer='morph_0')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [3]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_0', ud_layer_name, 'morph_0_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 1. No spelling suggestions + Vabamorf's analysis with disambiguation

In [4]:
# 'morph_1' == VabamorfTagger
from estnltk.taggers import VabamorfTagger

vm_tagger = VabamorfTagger(output_layer='morph_1',
                           input_words_layer='words')
for text in loaded_texts:
    vm_tagger.tag( text )

In [5]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_1', ud_layer_name, 'morph_1_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### Results: Vabamorf's analysis and disambiguation (baseline: no spelling suggestions)

In [6]:
from ewtb_ud_utils import eval_disambiguation_of_all_words

# get training part of the corpus
#evaluation_texts = [text for text in loaded_texts if 'train' in text.meta['file']]

# ... or evaluate on all texts
evaluation_texts = loaded_texts

eval_disambiguation_of_all_words( evaluation_texts, 'morph_0_diff_layer', 'morph_0_diff_layer', 'morph_1_diff_layer' )

 Words that needed disambiguation:                9852 / 27286
   Incorrectly disambiguated:                     867 / 9852    8.80%
   Correctly disambiguated:                       6520 / 9852    66.18%
   Disambiguation attempts:                       7387 / 9852    74.98%

   Correct words (including undisambiguated):     8938 / 9852    90.72%
 VM words alignable to UD morph words (before disamb):  26458 / 27286    96.97%
 VM words alignable to UD morph words  (after disamb):  25544 / 27286    93.62%


---

## Vabamorf's analysis and disambiguation with spelling suggestions

### Reload the data

In [7]:
eval_data_dir = 'UD_Estonian-EWT-master'

import os, os.path
from ewtb_ud_utils import load_EWTB_ud_file_with_corrections

# Load corpus files with corrections
ud_layer_name = 'ud_syntax'
loaded_texts  = []
for fname in os.listdir( eval_data_dir ):
    if fname.endswith('.conllu'):
        fpath = os.path.join( eval_data_dir, fname )
        text = load_EWTB_ud_file_with_corrections( fpath, ud_layer_name )
        text.meta['file'] = fname
        loaded_texts.append( text )

### Create VMSpellingSuggestionsTagger

Make a tagger that creates a special words layer containing spelling suggestions:

In [8]:
from estnltk import Text, Annotation, ElementaryBaseSpan
from estnltk.layer.layer import Layer
from estnltk.taggers import Tagger
from estnltk.vabamorf import morf as vm
from estnltk.taggers.morph_analysis.morf_common import _get_word_texts

class VMSpellingSuggestionsTagger(Tagger):
    '''Creates normalized_words layer which contains spelling suggestions from Vabamorf's spellchecker.'''
    conf_param = ['keep_original_word_text']
    output_attributes = []
    
    def __init__(self, words_layer='words', output_layer='normalized_words', keep_original_word_text=True):
        self.input_layers = [words_layer]
        self.output_layer = output_layer
        self.output_attributes = ('normalized_form',)
        self.keep_original_word_text = keep_original_word_text
    
    def _make_layer(self, text, layers, status):
        normalzed_words = Layer(name=self.output_layer,
                                attributes=self.output_attributes,
                                text_object=text,
                                ambiguous=True)
        words_layer = layers[self.input_layers[0]]
        for word in words_layer:
            if 'normalized_form' in words_layer.attributes:
                word_texts = _get_word_texts(word)
            else:
                word_texts = [word.text]
            suggestions = set()
            for word_text in word_texts:
                spell_check_result = vm.spellcheck([word_text], suggestions=True)
                # Check if we have a misspelled word with suggestions
                for item in spell_check_result:
                    if not item["spelling"] and len(item["suggestions"]) > 0:
                        for new_suggestion in item["suggestions"]:
                            if new_suggestion not in suggestions:
                                suggestions.add( new_suggestion )                
            if suggestions:
                if self.keep_original_word_text:
                    # (+) Pros of keeping original word text:
                    #     1) spellchecker may suggest that proper nouns, such as 'Rammstein' and 'Erasmuse',
                    #        are wrong, but morph analysis guesser can actually analyse these reasonably 
                    #        well;
                    #     2) spellchecker may suggest that nouns, such as 'krossikal' and 'reformarite',
                    #        are wrong, but morph analysis guesser can actually analyse these reasonably 
                    #        well;
                    #     3) spellchecker may suggest that adverbs, such as 'tegelt', are wrong, but morph 
                    #        analysis guesser can actually analyse these reasonably well;
                    #     4) spellchecker may suggest corrections to interjections, such as 'oih' and 'Mhh',
                    #        which actually need no corrections / normalizations in their lemmas;
                    # (-) Cons of keeping original word text:
                    #     Places where UD treebank's manual corrections are inconsistent or wrong may
                    #     become invisible, and will be counted as "correct matches", although they 
                    #     actually are not. Examples:
                    #     1) lowercase propernames, such as 'tallinnast' or 'iklat', will match, because 
                    #        their UD xpostag is 'S', although correct should be 'H';
                    #     2) adjectives, such as 'krõvisevaid' and 'kõkuvaid', will match, because their
                    #        UD lemmas have not been corrected ( 'krõvisevaid' => 'krõvisev', but correct 
                    #        is 'krõbisev'; 'kõkuvaid' => 'kõkuv', but is correct: 'kõikuv');
                    #     3) nouns, such as 'konsentratsioon', will match because of incorrect UD lemmas
                    #        ('konsentratsioon' => 'konsentratsioon', although correct is 'kontsentratsioon');
                    if word.text not in suggestions:
                        normalzed_words.add_annotation( word.base_span, normalized_form=word.text )
                for suggestion in suggestions:
                    normalzed_words.add_annotation( word.base_span, normalized_form=suggestion )
            else:
                normalzed_words.add_annotation( word.base_span, normalized_form=None )
        return normalzed_words


test_text = Text('Ma tahax teada assju.')
test_text.tag_layer(['words'])
VMSpellingSuggestionsTagger().tag(test_text).normalized_words

Layer(name='normalized_words', attributes=('normalized_form',), spans=SL[Span('Ma', [{'normalized_form': None}]),
Span('tahax', [{'normalized_form': 'tahax'}, {'normalized_form': 'tahaks'}, {'normalized_form': 'tahad'}, {'normalized_form': 'taha'}]),
Span('teada', [{'normalized_form': None}]),
Span('assju', [{'normalized_form': 'assju'}, {'normalized_form': 'asju'}, {'normalized_form': 'assjõu'}]),
Span('.', [{'normalized_form': None}])])

### Apply VMSpellingSuggestionsTagger on the input corpus

In [9]:
spelling_suggestor = VMSpellingSuggestionsTagger()
for text in loaded_texts:
    spelling_suggestor.tag( text )

### 0. No spelling suggestions + Vabamorf's analysis

In [10]:
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_0')
post_corrector = PostMorphAnalysisTagger(output_layer='morph_0')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [11]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_0', ud_layer_name, 'morph_0_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 1. Spelling suggestions + Vabamorf's analysis only

In [12]:
# 'morph_1' == VabamorfAnalyzer + PostMorphAnalysisTagger
from estnltk.taggers import VabamorfAnalyzer, PostMorphAnalysisTagger

vm_analyser = VabamorfAnalyzer(output_layer='morph_1',
                               input_words_layer=spelling_suggestor.output_layer)
post_corrector = PostMorphAnalysisTagger(output_layer='morph_1')
for text in loaded_texts:
    vm_analyser.tag( text )
    post_corrector.retag( text )

In [13]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_1', ud_layer_name, 'morph_1_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### 2. Spelling suggestions + Vabamorf's analysis with disambiguation

In [14]:
# 'morph_2' == VabamorfTagger
from estnltk.taggers import VabamorfTagger

vm_tagger = VabamorfTagger(output_layer='morph_2',
                           input_words_layer=spelling_suggestor.output_layer)
for text in loaded_texts:
    vm_tagger.tag( text )

In [15]:
from ewtb_ud_utils import VM2UDMorphFullDiffTagger
vm2ud_diff_tagger = VM2UDMorphFullDiffTagger('morph_2', ud_layer_name, 'morph_2_diff_layer')
# Find differences
for text in loaded_texts:
    vm2ud_diff_tagger.tag(text)

### Results: Vabamorf's analysis and disambiguation after spelling suggestions

In [16]:
from ewtb_ud_utils import eval_disambiguation_of_all_words

# get training part of the corpus
#evaluation_texts = [text for text in loaded_texts if 'train' in text.meta['file']]

# ... or evaluate on all texts
evaluation_texts = loaded_texts

eval_disambiguation_of_all_words( evaluation_texts, 'morph_0_diff_layer', 'morph_1_diff_layer', 'morph_2_diff_layer' )

 Words that needed disambiguation:                10158 / 27286
   Incorrectly disambiguated:                     958 / 10158    9.43%
   Correctly disambiguated:                       6488 / 10158    63.87%
   Disambiguation attempts:                       7446 / 10158    73.30%

   Correct words (including undisambiguated):     9081 / 10158    89.40%
 VM words alignable to UD morph words (before disamb):  26637 / 27286    97.62%
 VM words alignable to UD morph words  (after disamb):  25560 / 27286    93.67%


And get a fine-grained statistics about disambiguation quality only on normalized words:

In [17]:
from ewtb_ud_utils import eval_disambiguation_of_normalized_words

# get training part of the corpus
#evaluation_texts = [text for text in loaded_texts if 'train' in text.meta['file']]

# ... or evaluate on all texts
evaluation_texts = loaded_texts

eval_disambiguation_of_normalized_words( evaluation_texts, spelling_suggestor.output_layer, 'morph_1_diff_layer', 'morph_2_diff_layer' )

 Only words with normalizations (spelling suggestions):  637 / 27286
   Correctly analysed (without disambiguation):          484 / 637    75.98%
       - Correctly disambiguated:                        288 / 637    45.21%
       - Incorrectly disambiguated:                      196 / 637    30.77%
   Word's analyses cannot be matched to UD word's:       153 / 637    24.02%



---

## Summary

    Measurements made on training & test parts of the EWTB corpus
    
    A) Vabamorf's analysis & disambiguation (baseline: no spelling corrections)
    ================================================================================
     Words that needed disambiguation:                9852 / 27286
       Incorrectly disambiguated:                     867 / 9852      8.80%
       Correctly disambiguated:                       6520 / 9852    66.18%
       Disambiguation attempts:                       7387 / 9852    74.98%

       Correct words (including undisambiguated):     8938 / 9852    90.72%
    ================================================================================
     VM words alignable to UD morph words (before disamb):  26458 / 27286    96.97%
     VM words alignable to UD morph words  (after disamb):  25544 / 27286    93.62%

    B) Vabamorf's analysis & disambiguation on words with spelling suggestions    
    ================================================================================
     Words that needed disambiguation:                10158 / 27286
       Incorrectly disambiguated:                     958 / 10158      9.43%
       Correctly disambiguated:                       6488 / 10158    63.87%
       Disambiguation attempts:                       7446 / 10158    73.30%

       Correct words (including undisambiguated):     9081 / 10158    89.40%
    ================================================================================
     VM words alignable to UD morph words (before disamb):  26637 / 27286    97.62%
     VM words alignable to UD morph words  (after disamb):  25560 / 27286    93.67%

     ================================================================================
      Only words with normalizations (spelling suggestions):  637 / 27286
        Correctly analysed (without disambiguation):          484 / 637    75.98%
            - Correctly disambiguated:                        288 / 637    45.21%
            - Incorrectly disambiguated:                      196 / 637    30.77%
        Word's analyses cannot be matched to UD word's:       153 / 637    24.02%
      ================================================================================